## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Hobart,AU,-42.8794,147.3294,48.06,70,40,1.99,scattered clouds,2021-10-09 15:25:10
1,1,Bluff,NZ,-46.6000,168.3333,42.80,92,68,8.84,broken clouds,2021-10-09 15:24:04
2,2,Terme,TR,41.2092,36.9739,64.96,88,2,6.15,clear sky,2021-10-09 15:27:18
3,3,Thompson,CA,55.7435,-97.8558,32.16,100,90,2.30,fog,2021-10-09 15:25:44
4,4,Chanderi,IN,24.7167,78.1333,79.18,57,68,7.47,broken clouds,2021-10-09 15:27:19


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
2,2,Terme,TR,41.2092,36.9739,64.96,88,2,6.15,clear sky,2021-10-09 15:27:18
4,4,Chanderi,IN,24.7167,78.1333,79.18,57,68,7.47,broken clouds,2021-10-09 15:27:19
6,6,Albany,US,42.6001,-73.9662,64.17,76,100,1.99,overcast clouds,2021-10-09 15:22:19
8,8,Rikitea,PF,-23.1203,-134.9692,74.17,80,7,17.16,clear sky,2021-10-09 15:23:51
9,9,Vaini,TO,-21.2000,-175.2000,68.16,100,75,6.91,thunderstorm,2021-10-09 15:23:46
10,10,Mahebourg,MU,-20.4081,57.7000,73.69,73,40,14.97,light rain,2021-10-09 15:23:55
13,13,Aden,YE,12.7794,45.0367,86.76,74,22,10.78,few clouds,2021-10-09 15:26:25
14,14,Basoko,CD,1.2391,23.6160,88.16,55,93,0.96,overcast clouds,2021-10-09 15:27:21
15,15,Puerto Ayora,EC,-0.7393,-90.3518,73.36,81,99,10.98,overcast clouds,2021-10-09 15:24:12
16,16,Kavieng,PG,-2.5744,150.7967,81.03,80,100,8.66,overcast clouds,2021-10-09 15:23:43


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                378
City                   378
Country                375
Lat                    378
Lng                    378
Max Temp               378
Humidity               378
Cloudiness             378
Wind Speed             378
Current Description    378
Date                   378
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                375
City                   375
Country                375
Lat                    375
Lng                    375
Max Temp               375
Humidity               375
Cloudiness             375
Wind Speed             375
Current Description    375
Date                   375
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Terme,TR,64.96,clear sky,41.2092,36.9739,
4,Chanderi,IN,79.18,broken clouds,24.7167,78.1333,
6,Albany,US,64.17,overcast clouds,42.6001,-73.9662,
8,Rikitea,PF,74.17,clear sky,-23.1203,-134.9692,
9,Vaini,TO,68.16,thunderstorm,-21.2000,-175.2000,
10,Mahebourg,MU,73.69,light rain,-20.4081,57.7000,
13,Aden,YE,86.76,few clouds,12.7794,45.0367,
14,Basoko,CD,88.16,overcast clouds,1.2391,23.6160,
15,Puerto Ayora,EC,73.36,overcast clouds,-0.7393,-90.3518,
16,Kavieng,PG,81.03,overcast clouds,-2.5744,150.7967,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, hotel in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = hotel["Lat"]
    lng = hotel["Lng"]
    params["location"]= f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   375
Country                375
Max Temp               375
Current Description    375
Lat                    375
Lng                    375
Hotel Name             375
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current description</dt><dd>{Current Description}</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))